# Prerequisites

Required CSVS:

data/conflict.csv - raw conflict data

data/climate-complete-noaa.csv - raw climate data

data/clean_fews.csv - transformed food data from FEWSNET

data/clean_food.csv - transformed food data from WFP

data/clean_ipc.csv - transformed IPC data from FSNAU


For the last 3 csvs, see other notebook on transformation

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pystan
import datetime as dt
import scipy.special
import os
os.getcwd()[30:]

'. Cambridge\\1. Work\\3. Courses\\1B\\Group Project\\BackendProto\\notebooks'

# Model

There are three response vectors, $A_i$,$B_i$, $C_i$, where $A$ is the % of population in IPC Phase 2 and $B$ is the % of population in IPC Phase 3 and $C_i$ is the % of the population in IPC Phase 4. We have $N$ points of data, and $K$ features to estimate the coefficients of.

$$\mu_{A_i} = \text{inv_logit}(\alpha_{A} + \sum_{X\in\{A,B,C\}}(\beta_{A,X} * \text{logit}(X_{i-1})) + \sum_k(\text{coeffs}_{A,k} * \text{feats}_{i,k}))$$

$$A_i \sim \text{Beta_prop}(\mu_{A_i}, \kappa_A)$$

$$\mu_{B_i} = \text{inv_logit}(\alpha_{B} + \sum_{X\in\{A,B,C\}}(\beta_{B,X} * \text{logit}(X_{i-1})) + \sum_k(\text{coeffs}_{B,k} * \text{feats}_{i,k}))$$

$$B_i \sim \text{Beta_prop}(\mu_{B_i}, \kappa_B)$$

$$\mu_{C_i} = \text{inv_logit}(\alpha_{C} + \sum_{X\in\{A,B,C\}}(\beta_{C,X} * \text{logit}(X_{i-1})) + \sum_k(\text{coeffs}_{C,k} * \text{feats}_{i,k}))$$

$$C_i \sim \text{Beta_prop}(\mu_{C_i}, \kappa_C)$$

Beta_prop is a variant of the Beta distribution, with parameters $\mu$, which is the mean, and $\kappa$, which is the precision, or the inverse of the variance, where a high $\kappa$ implies a low variance.

To transform $\mu$ and $\kappa$ back into the standard parameters $\alpha$ and $\beta$:
$$ \alpha = \mu\kappa $$
$$ \beta = (1-\mu)\kappa $$

logit is the logit function $\text{logit(p)} = \log(\frac{p}{1-p})$, and maps $(0,1)$ to $(-\infty, +\infty)$, 
and inv_logit is the inverse function $\text{inv_logit(x)} = \frac{e^x}{e^x + 1}$

All the parameters and data ending with _2 refers to IPC Phase 2, _3 to IPC Phase 3, and _4 to IPC Phase 4.

In [3]:
beta_model_code = '''
data {
    int<lower=0> N;
    int<lower=0> K;
    matrix[N,K] feats;
    vector[N] response_2;
    vector[N] response_3;
    vector[N] response_4;
}
transformed data {
    vector[N] logit_response_2;
    vector[N] logit_response_3;
    vector[N] logit_response_4;
    
    logit_response_2 = logit(response_2);
    logit_response_3 = logit(response_3);
    logit_response_4 = logit(response_4);
}
parameters{
    real alpha_2;
    vector[3] beta_2;
    vector[K] coeffs_2;
    real<lower = 100> kappa_2;
    
    real alpha_3;
    vector[3] beta_3;
    vector[K] coeffs_3;
    real<lower = 100> kappa_3;
    
    real alpha_4;
    vector[3] beta_4;
    vector[K] coeffs_4;
    real<lower = 100> kappa_4;
}

model{
    vector[N] mus_2;
    vector[N] mus_3;
    vector[N] mus_4;
    
    mus_2[2:N] = inv_logit(alpha_2 + beta_2[1]*logit_response_2[1:(N-1)] + beta_2[2]*logit_response_3[1:(N-1)] + beta_2[3]*logit_response_4[1:(N-1)] + feats[2:N]*coeffs_2);
    response_2[2:N] ~ beta_proportion(mus_2[2:N], kappa_2);
    
    mus_3[2:N] = inv_logit(alpha_3 + beta_3[1]*logit_response_2[1:(N-1)] + beta_3[2]*logit_response_3[1:(N-1)] + beta_3[3]*logit_response_4[1:(N-1)] + feats[2:N]*coeffs_3);
    response_3[2:N] ~ beta_proportion(mus_3[2:N], kappa_3);
    
    mus_4[2:N] = inv_logit(alpha_4 + beta_4[1]*logit_response_2[1:(N-1)] + beta_4[2]*logit_response_3[1:(N-1)] + beta_4[3]*logit_response_4[1:(N-1)] + feats[2:N]*coeffs_4);
    response_4[2:N] ~ beta_proportion(mus_4[2:N], kappa_4);
}
'''
beta_model = pystan.StanModel(model_code=beta_model_code, model_name="beta_model")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL beta_model_c1f83952f869caec67ffdfc8087592cc NOW.


In [10]:
#Export models
REGIONS = ['Awdal', 'Bakool', 'Banadir', 'Bari', 'Bay', 'Galgaduud', 'Gedo', 'Hiraan', 'Lower Juba', 'Lower Shabelle', 'Middle Juba', 'Middle Shabelle', 'Mudug', 'Nugaal', 'Sanaag', 'Sool', 'Togdheer', 'Woqooyi Galbeed']
if (True):
    for region in REGIONS[9:]:
        datasets = extract_features(region)
        if(datasets == None):
            print("{} has too little data to fit model".format(region))
        else:
            print("{}".format(region))
            model = fit_model(datasets, 0)
            plot_model(datasets, model)
            model.to_dataframe().to_csv("model/{}.csv".format(region), index=False)

#Saves feature names
rows = []

for region in REGIONS:
    datasets = extract_features(region)
    if (datasets != None):
        rows.append(dict(region=region, feature_names=list(datasets.values())[0]['feature_names']))
feature_names_df = pd.DataFrame(rows)
feature_names_df.to_csv("model/feature-names.csv", index=False)


FileNotFoundError: [Errno 2] File data/clean_food.csv does not exist: 'data/clean_food.csv'

In [6]:
#Helper Functions for extract_features
def get_days(t1, t2):
    y1, y2 = int(t1), int(t2)
    if(y1 == y2):
        return int((t2-t1)*(366 if y2%4==0 else 365))
    else:
        return (366-274)

def get_prev_quarter(t):
    year = t//1
    days = int(round((t-year) * (366 if year%4==0 else 365)))
    if(days == 90 or days == 91):
        return year
    elif(days == 181):
        return year + 90/365
    elif(days == 182):
        return year + 91/366
    elif(days == 273):
        return year + 181/365
    elif(days == 274):
        return year + 182/366
    elif(days == 0):
        return ((year-1)+274/366) if ((year-1)%4==0) else ((year-1) + 273/365)
    else:
        print(days, t)


def strdate(s):
    date = dt.datetime.strptime(s, '%d %B %Y')
    year = date.date().year
    return year + (date.date().timetuple().tm_yday-1)/ (366 if year%4==0 else 365)
MONTHS = list(np.cumsum([0, 31,28,31,30,31,30,31,31,30,31,30,31]))
LEAPS = list(np.cumsum([0, 31,29,31,30,31,30,31,31,30,31,30,31]))

def dateToFloat(d):
    year= d//10000
    mo = (d%10000)//100 - 1
    day = d%100 - 1
    if(year%4==0):
        return year+(LEAPS[mo]+day)/366
    else:
        return year+(MONTHS[mo]+day)/365

In [7]:
'''
extract_features(region):

returns datasets, a dictionary where the keys are dates and the values are dictionaries.

Each dictionary has the following keys and values:
'features' - list of feature values
'feature_names' - name of each feature
'food' - a Pandas dataframe of food information
'conflict' - a Pandas dataframe of conflict information
'weather' - a Pandas dataframe of weather information
'ipc' - a dictionary:
    'p2perc': Proportion in IPC Phase 2
    'p3perc': Proportion in IPC Phase 3
    'p4perc': Proportion in IPC Phase 4
    
    
Some regions have insufficient data, they will return None if extract_features is called
'''

def extract_features(region):
    #CONSTANTS
    STARTTIME = 2016
    WANTEDFEWSFOOD = {'Cowpeas (Red)'}
    WANTEDSTATIONS = {'EGAL INTL'}
    
     #Transform food data:
    food_df = pd.read_csv('data/clean_food.csv')
    food_df = food_df[food_df.Region.eq(region) & food_df.Date.ge(STARTTIME-1)]
    trimmed_food_df = food_df[['Date', 'Item', 'Price', 'Market']]

    #Constrained by food data dates, get the earliest and latest dates here:
    earliest_date = min(trimmed_food_df.Date.values)
    latest_date = max(trimmed_food_df.Date.values)

    #Extract FEWSNET food data
    ffood_df = pd.read_csv('data/clean_fews.csv')
    ffood_df = ffood_df[ffood_df.Date.ge(earliest_date) & ffood_df.Date.le(latest_date) & ffood_df.Item.isin(WANTEDFEWSFOOD) & ffood_df.Region.eq(region)]
    ffood_df = ffood_df[['Date', 'Item', 'Price', 'Market']]

    #Extract conflict data
    conflict_df = pd.read_csv("data/conflict.csv")
    conflict_df = conflict_df[['event_date', 'admin1', 'fatalities']].rename(columns={'admin1':'region'})
    conflict_df['date'] = conflict_df.event_date.apply(strdate)
    trimmed_conflict_df = conflict_df[conflict_df.date.ge(STARTTIME-1) & conflict_df.region.eq(region)].rename(columns={
        "date": "Date",
        "region": "Region",
        "fatalities": "Fatalities"
    }).sort_values(by='Date')[['Date','Region','Fatalities']]

    #Extract IPC data
    ipc_df = pd.read_csv('data/clean-ipc.csv')
    trimmed_ipc_df = ipc_df[ipc_df.time.ge(STARTTIME) & ipc_df.region.eq(region)]


    #Extract weather data
    weather_df = pd.read_csv('data/climate-complete-noaa.csv', usecols = [0,3,4])
    weather_df = weather_df[weather_df['Station Name'].isin(WANTEDSTATIONS)]
    weather_df['Date'] = weather_df['Date'].apply(dateToFloat)
    weather_df = weather_df[weather_df.Date.ge(STARTTIME-1)]
    trimmed_weather_df = weather_df.rename(columns={'Station Name':'Station', 'Mean Temperature':'Temperature'})

    all_dates = sorted(list(set(trimmed_ipc_df.time.values)))
    valid_dates = []
    for date in all_dates:
        if(date - .125 >= earliest_date and date <= latest_date + .125):
            valid_dates.append(date)

    if(len(valid_dates)<=12):
        #Insufficient data, trying to fit with data will give an over-fitted model
        return None
    
    
    datasets = dict()
    for date in valid_dates:
        prev_q = get_prev_quarter(date)
        dataset = dict()
        features = []
        feature_names = []
        dataset['ipc']= trimmed_ipc_df.loc[trimmed_ipc_df.time.eq(date), ['p2perc', 'p3perc', 'p4perc']].to_dict('records')[0]

        food_data = dict()
        items = sorted(set(trimmed_food_df.Item.values))
        for item in items:
            food_data[item] = trimmed_food_df[trimmed_food_df.Date.ge(prev_q) & trimmed_food_df.Date.lt(date) & trimmed_food_df.Item.eq(item)]
            features.append(np.mean(food_data[item].Price.values)/1e4)
            feature_names.append("{} - {}".format(food_data[item].Market.values[0], item))

        for food in WANTEDFEWSFOOD:
            food_data[food] = ffood_df[ffood_df.Date.ge(prev_q) & ffood_df.Date.lt(date) & ffood_df.Item.eq(food)]
            feature = np.mean(food_data[food].Price.values)/1e4
            if(not np.isnan(feature)):
                features.append(np.mean(food_data[food].Price.values)/1e4)
                feature_names.append("{} - {}".format(food_data[item].Market.values[0], item))
        
        dataset['food'] = food_data

        conflict_data = trimmed_conflict_df[trimmed_conflict_df.Date.ge(prev_q)&trimmed_conflict_df.Date.lt(date)]
        features.append(np.sum(conflict_data.Fatalities.values)/get_days(prev_q,date))
        feature_names.append("Fatalities due to Conflict")
        dataset['conflict'] = conflict_data

        weather_data = dict()
        for station in WANTEDSTATIONS:
            weather_data[station] = trimmed_weather_df[trimmed_weather_df.Date.ge(prev_q) & trimmed_weather_df.Date.lt(date) & trimmed_weather_df.Station.eq(station)][['Date','Temperature']]
            if(len(weather_data[station])<30):
                weather_data[station] = trimmed_weather_df[trimmed_weather_df.Date.ge(prev_q+1) & trimmed_weather_df.Date.lt(date+1) & trimmed_weather_df.Station.eq(station)][['Date','Temperature']]
            features.append(np.mean(weather_data[station].Temperature.values)/1e2)
            feature_names.append("Temperature")
        dataset['weather']=weather_data

        
        dataset['features']=features
        dataset['feature_names'] = feature_names
        datasets[date] = dataset
    return datasets

In [8]:
'''
fit_model(datasets, holdout)
Attempts to fit datasets, but ignoring out the last <holdout> dates for evaluation
Returns a StanFit4Model object
'''
def fit_model(datasets, holdout):
    train_dates = sorted(list (datasets.keys()))[:-holdout] if (holdout>0) else sorted(list (datasets.keys()))
    nFeatures = len(datasets[train_dates[0]]['features'])
    features = [datasets[date]['features'] for date in train_dates]
    response_2 = [max(datasets[date]['ipc']['p2perc'], 1e-5) for date in train_dates]
    response_3 = [max(datasets[date]['ipc']['p3perc'], 1e-5)  for date in train_dates]
    response_4 = [max(datasets[date]['ipc']['p4perc'], 1e-5)  for date in train_dates]
    famine_model_data = dict(
        N = len(train_dates),
        K = nFeatures,
        feats = features,
        response_2 = response_2,
        response_3 = response_3,
        response_4 = response_4
    )
    result = beta_model.sampling(data=famine_model_data, iter=3000, control = dict(max_treedepth=12, adapt_delta=0.8))
    return result

In [9]:
'''
plot_model(datasets, model)
Plots the predicted and actual IPC Phase proportion values based on the datasets and the StanFit4Model object
'''

def plot_model(datasets, model):
    nFeatures = len(datasets[list(datasets.keys())[0]]['features'])
    all_coeffs = list(map(lambda x: sum(x)/len(x), model.get_posterior_mean()))
    al_2, be_2, co_2, k_2 = all_coeffs[0],all_coeffs[1:4],all_coeffs[4:4+nFeatures],all_coeffs[4+nFeatures]
    al_3, be_3, co_3, k_3 = all_coeffs[5+nFeatures],all_coeffs[6+nFeatures:9+nFeatures],all_coeffs[9+nFeatures:9+2*nFeatures],all_coeffs[9+2*nFeatures]
    al_4, be_4, co_4, k_4 = all_coeffs[10+2*nFeatures],all_coeffs[11+2*nFeatures:14+2*nFeatures],all_coeffs[14+2*nFeatures:14+3*nFeatures],all_coeffs[14+3*nFeatures]
    
    #Helper function to produce the predicted values
    def generate(al_2, al_3, al_4, be_2, be_3, be_4, co_2, co_3, co_4, datasets):
        times = sorted(list(datasets.keys()))
        pred_ipc2_logit = [scipy.special.logit(max(datasets[times[0]]['ipc']['p2perc'], 1e-5))]
        pred_ipc3_logit = [scipy.special.logit(max(datasets[times[0]]['ipc']['p3perc'], 1e-5))]
        pred_ipc4_logit = [scipy.special.logit(max(datasets[times[0]]['ipc']['p4perc'], 1e-5))]
        
        for time in times[1:]:
            new_ipc_2a = al_2 + be_2[0]*pred_ipc2_logit[-1] + be_2[1]*pred_ipc3_logit[-1] + be_2[2]*pred_ipc4_logit[-1] + sum(np.multiply(co_2, datasets[time]['features']))
            new_ipc_3a = al_3 + be_3[0]*pred_ipc2_logit[-1] + be_3[1]*pred_ipc3_logit[-1] + be_3[2]*pred_ipc4_logit[-1] + sum(np.multiply(co_3, datasets[time]['features']))
            new_ipc_4a = al_4 + be_4[0]*pred_ipc2_logit[-1] + be_4[1]*pred_ipc3_logit[-1] + be_4[2]*pred_ipc4_logit[-1] + sum(np.multiply(co_4, datasets[time]['features']))
            pred_ipc2_logit.append(new_ipc_2a)
            pred_ipc3_logit.append(new_ipc_3a)
            pred_ipc4_logit.append(new_ipc_4a)
        
        pred_ipc2 = list(map(scipy.special.expit, pred_ipc2_logit))
        pred_ipc3 = list(map(scipy.special.expit, pred_ipc3_logit))
        pred_ipc4 = list(map(scipy.special.expit, pred_ipc4_logit))
        return (pred_ipc2,pred_ipc3, pred_ipc4)
    
    (pred_ipc2, pred_ipc3, pred_ipc4) = generate(al_2, al_3, al_4, be_2, be_3, be_4, co_2, co_3, co_4, datasets)
    print("IPC PHASE 2")
    times = sorted(list(datasets.keys()))
    gold_ipc2 = [datasets[date]['ipc']['p2perc'] for date in times]
    
    plt.plot(times, gold_ipc2, marker='o', color='gold')
    plt.plot(times,pred_ipc2, marker='+', color='red')
    plt.show()

    print("IPC PHASE 3")
    gold_ipc3 = [datasets[date]['ipc']['p3perc'] for date in times]
    
    plt.plot(times, gold_ipc3, marker='o', color='gold')
    plt.plot(times,pred_ipc3, marker='+', color='blue')
    plt.show()
    
    print("IPC PHASE 4")
    gold_ipc4 = [datasets[date]['ipc']['p4perc'] for date in times]
    
    plt.plot(times, gold_ipc4, marker='o', color='gold')
    plt.plot(times,pred_ipc4, marker='+', color='brown')
    plt.show()
    